In [1]:
#installation of libraries
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt

In [2]:
#calling the dataset
df = pd.read_csv("online_retail_II.csv")

In [3]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [4]:
df.groupby("Description").agg({"Quantity":"sum"}).sort_values("Quantity", ascending = False).head()

,Quantity
Description,
WORLD WAR 2 GLIDERS ASSTD DESIGNS,108545
WHITE HANGING HEART T-LIGHT HOLDER,93050
ASSORTED COLOUR BIRD ORNAMENT,81306
JUMBO BAG RED RETROSPOT,78090
BROCADE RING PURSE,70700


In [5]:
df["Invoice"].nunique()

53628

In [6]:
df.sort_values("Price", ascending = False).head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
748142,C556445,M,Manual,-1,2011-06-10 15:31:00,38970.00,15098.0,United Kingdom
241827,512771,M,Manual,1,2010-06-17 16:53:00,25111.09,NaN,United Kingdom
241824,C512770,M,Manual,-1,2010-06-17 16:52:00,25111.09,17399.0,United Kingdom
320581,C520667,BANK CHARGES,Bank Charges,-1,2010-08-27 13:42:00,18910.69,NaN,United Kingdom
1050063,C580605,AMAZONFEE,AMAZON FEE,-1,2011-12-05 11:36:00,17836.46,NaN,United Kingdom


In [7]:
#top 5 countries with the highest number of orders
df["Country"].value_counts().head()

United Kingdom    981330
EIRE               17866
Germany            17624
France             14330
Netherlands         5140
Name: Country, dtype: int64

In [8]:
#total spending was added as a column
df['TotalPrice'] = df['Price']*df['Quantity']

In [9]:
#which countries did we get the most income from
df.groupby("Country").agg({"TotalPrice":"sum"}).sort_values("TotalPrice", ascending = False).head()

,TotalPrice
Country,
United Kingdom,1.638258e+07
EIRE,6.155196e+05
Netherlands,5.485249e+05
Germany,4.179886e+05
France,3.281918e+05


Data Preparation

In [10]:
df["InvoiceDate"].min() #oldest shopping date

'2009-12-01 07:45:00'

In [11]:
df["InvoiceDate"].max() #newest shopping date

'2011-12-09 12:50:00'

In [20]:
today = pd.datetime(2012,1,1) 
today

C:\Users\Luan Lima\AppData\Local\Temp\ipykernel_21000\3518223459.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  today = pd.datetime(2012,1,1)


datetime.datetime(2012, 1, 1, 0, 0)

In [12]:
#changing the data type of the order date
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [13]:
df = df[df['Quantity'] > 0]
df = df[df['TotalPrice'] > 0]

In [14]:
df.dropna(inplace = True)

In [15]:
df.shape

(805549, 9)

In [16]:
df.describe([0.01,0.05,0.10,0.25,0.50,0.75,0.90,0.95, 0.99]).T

,count,mean,std,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max
Quantity,805549.0,13.290522,143.634088,1.000,1.00,1.00,1.00,2.00,5.00,12.00,24.00,36.0,128.00,80995.0
Price,805549.0,3.206561,29.199173,0.001,0.29,0.42,0.55,1.25,1.95,3.75,6.75,8.5,14.95,10953.5
Customer ID,805549.0,15331.954970,1696.737039,12346.000,12422.00,12681.00,12979.00,13982.00,15271.00,16805.00,17716.00,17913.0,18204.00,18287.0
TotalPrice,805549.0,22.026505,224.041928,0.001,0.55,1.25,2.08,4.95,11.85,19.50,35.40,67.5,201.60,168469.6


Finding RFM Score

In [17]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.0


In [18]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 805549 entries, 0 to 1067370
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      805549 non-null  object        
 1   StockCode    805549 non-null  object        
 2   Description  805549 non-null  object        
 3   Quantity     805549 non-null  int64         
 4   InvoiceDate  805549 non-null  datetime64[ns]
 5   Price        805549 non-null  float64       
 6   Customer ID  805549 non-null  float64       
 7   Country      805549 non-null  object        
 8   TotalPrice   805549 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 61.5+ MB


In [21]:
# finding Recency and Monetary values.
df_x = df.groupby('Customer ID').agg({'TotalPrice': lambda x: x.sum(), #monetary value
                                        'InvoiceDate': lambda x: (today - x.max()).days}) #recency value
#x.max()).days; last shopping date of customers

In [22]:
df_x.head()

,TotalPrice,InvoiceDate
Customer ID,,
12346.0,77556.46,347
12347.0,5633.32,24
12348.0,2019.40,97
12349.0,4428.69,40
12350.0,334.40,332


In [23]:
df_y = df.groupby(['Customer ID','Invoice']).agg({'TotalPrice': lambda x: x.sum()})
df_z = df_y.groupby('Customer ID').agg({'TotalPrice': lambda x: len(x)}) 
#finding the frequency value per capita

In [26]:
df_y.head(5)

TotalPrice
Customer ID Invoice            
12346.0     491725         45.0
            491742         22.5
            491744         22.5
            492718         22.5
            492722          1.0

In [27]:
df_z.head(5)

,TotalPrice
Customer ID,
12346.0,12
12347.0,8
12348.0,5
12349.0,4
12350.0,1


In [28]:
rfm_table= pd.merge(df_x,df_z, on='Customer ID')

In [29]:
rfm_table.rename(columns= {'InvoiceDate': 'Recency',
                          'TotalPrice_y': 'Frequency',
                          'TotalPrice_x': 'Monetary'}, inplace= True)

In [30]:
rfm_table.head()

,Monetary,Recency,Frequency
Customer ID,,,
12346.0,77556.46,347,12
12347.0,5633.32,24,8
12348.0,2019.40,97,5
12349.0,4428.69,40,4
12350.0,334.40,332,1


Table Comment: The customer with the 12346th Customer ID made a purchase of £ 77.556. The last purchase of this customer is about 1 year ago (347 days) and the number of purchases is 12.

In [31]:
rfm_table['RecencyScore'] = pd.qcut(rfm_table['Recency'],5,labels=[5,4,3,2,1])
rfm_table['FrequencyScore'] = pd.qcut(rfm_table['Frequency'].rank(method="first"),5,labels=[1,2,3,4,5])
rfm_table['MonetaryScore'] = pd.qcut(rfm_table['Monetary'],5,labels=[1,2,3,4,5])

In [32]:
rfm_table.head()

,Monetary,Recency,Frequency,RecencyScore,FrequencyScore,MonetaryScore
Customer ID,,,,,,
12346.0,77556.46,347,12,2,5,5
12347.0,5633.32,24,8,5,4,5
12348.0,2019.40,97,5,3,4,4
12349.0,4428.69,40,4,5,3,5
12350.0,334.40,332,1,2,1,2


In [33]:
(rfm_table['RecencyScore'].astype(str) + 
 rfm_table['FrequencyScore'].astype(str) + 
 rfm_table['MonetaryScore'].astype(str)).head()

Customer ID
12346.0    255
12347.0    545
12348.0    344
12349.0    535
12350.0    212
dtype: object

In [34]:
rfm_table["RFM_SCORE"] = rfm_table['RecencyScore'].astype(str) + rfm_table['FrequencyScore'].astype(str) + rfm_table['MonetaryScore'].astype(str)

In [35]:
rfm_table.head()

,Monetary,Recency,Frequency,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
Customer ID,,,,,,,
12346.0,77556.46,347,12,2,5,5,255
12347.0,5633.32,24,8,5,4,5,545
12348.0,2019.40,97,5,3,4,4,344
12349.0,4428.69,40,4,5,3,5,535
12350.0,334.40,332,1,2,1,2,212


In [36]:
rfm_table.describe().T

,count,mean,std,min,25%,50%,75%,max
Monetary,5878.0,3018.616737,14737.731040,2.95,348.7625,898.915,2307.09,608821.65
Recency,5878.0,222.866791,209.353961,22.00,47.0000,117.000,401.00,760.00
Frequency,5878.0,6.289384,13.009406,1.00,1.0000,3.000,7.00,398.00


In [37]:
rfm_table[rfm_table["RFM_SCORE"] == "555"].head()

,Monetary,Recency,Frequency,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
Customer ID,,,,,,,
12362.0,5356.23,25,11,5,5,5,555
12395.0,5067.27,41,15,5,5,5,555
12417.0,6816.91,25,20,5,5,5,555
12433.0,20581.26,22,10,5,5,5,555
12437.0,12683.40,23,39,5,5,5,555


In [38]:
rfm_table[rfm_table["RFM_SCORE"] == "111"].head()

,Monetary,Recency,Frequency,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
Customer ID,,,,,,,
12387.0,143.94,437,1,1,1,1,111
12392.0,234.75,613,1,1,1,1,111
12400.0,205.25,436,1,1,1,1,111
12404.0,63.24,704,1,1,1,1,111
12416.0,202.56,678,1,1,1,1,111


In [39]:
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At Risk',
    r'[1-2]5': 'Can\'t Lose',
    r'3[1-2]': 'About to Sleep',
    r'33': 'Need Attention',
    r'[3-4][4-5]': 'Loyal Customers',
    r'41': 'Promising',
    r'51': 'New Customers',
    r'[4-5][2-3]': 'Potential Loyalists',
    r'5[4-5]': 'Champions'
}

In [43]:
rfm_table['Segment'] = rfm_table['RecencyScore'].astype(str) + rfm_table['FrequencyScore'].astype(str)

In [44]:
rfm_table.head()

,Monetary,Recency,Frequency,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE,Segment
Customer ID,,,,,,,,
12346.0,77556.46,347,12,2,5,5,255,25
12347.0,5633.32,24,8,5,4,5,545,54
12348.0,2019.40,97,5,3,4,4,344,34
12349.0,4428.69,40,4,5,3,5,535,53
12350.0,334.40,332,1,2,1,2,212,21


In [45]:
rfm_table['Segment'] = rfm_table['Segment'].replace(seg_map, regex=True)

In [46]:
rfm_table.head()

,Monetary,Recency,Frequency,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE,Segment
Customer ID,,,,,,,,
12346.0,77556.46,347,12,2,5,5,255,Can't Lose
12347.0,5633.32,24,8,5,4,5,545,Champions
12348.0,2019.40,97,5,3,4,4,344,Loyal Customers
12349.0,4428.69,40,4,5,3,5,535,Potential Loyalists
12350.0,334.40,332,1,2,1,2,212,Hibernating


In [42]:
rfm_table[["Segment", "Recency","Frequency","Monetary"]].groupby("Segment").agg(["mean","count"])

Recency        Frequency            Monetary      
                           mean count       mean count          mean count
Segment                                                                   
About to Sleep       128.618557   388   1.360825   388    531.969905   388
At Risk              394.942667   750   3.904000   750   1383.596253   750
Can't Lose           352.802817    71  15.929577    71   8355.677634    71
Champions             30.250000   852  19.214789   852  10795.520581   852
Hibernating          481.012484  1522   1.252300  1522    437.963135  1522
Loyal Customers       89.306016  1147   9.802964  1147   4199.728592  1147
Need Attention       135.241636   269   3.156134   269   1283.114015   269
New Customers         32.303571    56   1.000000    56    356.257857    56
Potential Loyalists   47.297335   713   2.589060   713   1155.427281   713
Promising             60.281818   110   1.000000   110    324.497000   110